<a href="https://colab.research.google.com/github/yukinaga/langchain/blob/main/section_4/02_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 翻訳アプリ
LangChainとStreamlitを使い、「翻訳アプリ」を構築します。  

## ライブラリのインストール
langchain-coreとlangchain-openaiをインストールします。  
また、UIを構築するための「Streamlit」、およびアプリの動作の確認に使用する「ngrok」をインストールします。  

In [ ]:
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok --quiet
!pip install langchain langchain-openai

## 翻訳アプリのコード
`%%writefile`のマジックコマンドを使って、翻訳アプリのコードを「app.py」に書き込みます。  
Streamlitでは、値を保持するために「st.session_state」を使います。  
https://docs.streamlit.io/library/api-reference/session-state

In [ ]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
import os
import secret_keys  # 外部ファイルにAPI keyを保存

os.environ["OPENAI_API_KEY"] = secret_keys.openai_api_key
chat = ChatOpenAI(model="gpt-3.5-turbo")

# プロンプトのテンプレート
system_template = (
    "あなたは、{source_lang} を {target_lang}に翻訳する優秀な翻訳アシスタントです。翻訳結果以外は出力しないでください。"
)
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

if "response" not in st.session_state:
    st.session_state["response"]= ""

# LLMとやりとりする関数
def communicate():
    text = st.session_state["user_input"]
    response = chat(
        chat_prompt.format_prompt(
            source_lang=source_lang, target_lang=target_lang, text=text
        ).to_messages()
    )
    st.session_state["response"] = response.content

# ユーザーインターフェイスの構築
st.title("翻訳アプリ")
st.write("LangChainを使った翻訳アプリです。")

options = ["日本語", "英語", "スペイン語", "ドイツ語", "フランス語", "中国語"]
source_lang = st.selectbox(label="翻訳元", options=options)
target_lang = st.selectbox(label="翻訳先", options=options)
st.text_input("翻訳する文章を入力してください。", key="user_input")
st.button("翻訳", type="primary", on_click=communicate)

if st.session_state["user_input"] != "":
    st.write("翻訳結果:")
    st.write(st.session_state["response"])

## OpenAIのAPI keyを設定
ChatGPT APIを使用するために必要な「API key」を設定します。  
`%%writefile`のマジックコマンドを使って、API keyを設定するコードを「secret_keys.py」に書き込みます。  
以下のコードの、  
`openai_api_key = "Your API key"`  
における  
`Your API key`の箇所を、自分のAPI keyに置き換えます。  
ChatGPTのAPI keyは、OpenAIのサイトで取得できます。   
https://platform.openai.com/account/api-keys


In [ ]:
%%writefile secret_keys.py

openai_api_key = "Your API key"

API keyの流出にはリスクがあります。  
他者に知られないように、慎重に扱ってください。

## ngrokのAuthtokenを設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [ ]:
!ngrok authtoken YourAuthtoken

ngrokのAuthtokenも、他者に知られないように慎重に扱ってください。

## アプリの起動
streamlitの`run`コマンドでアプリを起動します。


In [ ]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
ngrokの無料プランでは同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogle Colabのセッションを修了しましょう。  

In [ ]:
from pyngrok import ngrok

ngrok.kill()  # プロセスの修了
url = ngrok.connect("8501").public_url  # 接続

## 動作の確認

URLのhttpの部分をhttpsに変換する関数を設定します。

In [ ]:
def convert_http_to_https(url):
    if url.startswith("http://"):
        url = url.replace("http://", "https://", 1)
    return url

変換したurlを表示し、リンク先で翻訳アプリが動作することを確認します。

In [ ]:
print(convert_http_to_https(url))

動作確認後、OpenAIのサイトでAPIの使用量を確認してみましょう。  
https://platform.openai.com/account/usage